# Ejecución desde Jupyter de scripts FreeFem++

## 1. Llamada directa al intérprete FreeFem++

In [7]:
!FreeFem++-nw -ne keller-segel.edp -dt 0.001 -nt 2

-- FreeFem++ v  3.380001 (date Sat Feb  6 20:00:25 UTC 2016)
 Load: lg_fem lg_mesh lg_mesh3 eigenvalue 
 sizestack + 1024 =8000  ( 6976 )

Solving, iter: 1 (t=0.5)
  max(u): 3126.7
  max(v): 108.496
Solving, iter: 2 (t=1)
  max(u): 8754.46
  max(v): 383.326


## 2.- Ejecución de una shell bash

In [13]:
%%bash
nt=2
dt=0.001
FreeFem++-nw -ne keller-segel.edp -dt $dt -nt $nt

-- FreeFem++ v  3.380001 (date Sat Feb  6 20:00:25 UTC 2016)
 Load: lg_fem lg_mesh lg_mesh3 eigenvalue 
 sizestack + 1024 =8000  ( 6976 )

Solving, iter: 1 (t=0.001)
  max(u): 342.161
  max(v): 140.233
Solving, iter: 2 (t=0.002)
  max(u): 396.142
  max(v): 139.722


## 3.- Uso del módulo subprocess para ejecutar el intérprete FreeFem++

In [12]:
#!/usr/bin/env python
import subprocess

dt, T = 0.001, 1
command = "FreeFem++ -ne keller-segel.edp -dt {} -tf {}".format(dt,T).split()
print("command: ", command)

# Call a bash script
output = subprocess.check_output(command)  # subprocess.call(command)
print(output)

command:  ['FreeFem++', '-ne', 'keller-segel.edp', '-dt', '0.001', '-tf', '1']
b'-- FreeFem++ v  3.380001 (date Sat Feb  6 20:00:25 UTC 2016)\n Load: lg_fem lg_mesh lg_mesh3 eigenvalue \n sizestack + 1024 =8000  ( 6976 )\n\nSolving, iter: 1 (t=0.001)\n  max(u): 342.161\n  max(v): 140.233\nSolving, iter: 2 (t=0.002)\n  max(u): 396.142\n  max(v): 139.722\nSolving, iter: 3 (t=0.003)\n  max(u): 456.852\n  max(v): 139.269\nSolving, iter: 4 (t=0.004)\n  max(u): 524.813\n  max(v): 138.88\nSolving, iter: 5 (t=0.005)\n  max(u): 600.571\n  max(v): 138.56\nSolving, iter: 6 (t=0.006)\n  max(u): 684.72\n  max(v): 138.313\nSolving, iter: 7 (t=0.007)\n  max(u): 777.934\n  max(v): 138.144\nSolving, iter: 8 (t=0.008)\n  max(u): 881.004\n  max(v): 138.057\nSolving, iter: 9 (t=0.009)\n  max(u): 994.893\n  max(v): 138.057\nSolving, iter: 10 (t=0.01)\n  max(u): 1120.8\n  max(v): 138.147\n'


## 4. Captura en tiempo real la salida de FreeFem++

In [14]:
from subprocess import Popen, PIPE

def run(command):
    "Run command, yielding *realtime* output"
    process = Popen(command, stdout=PIPE, shell=True)
    while True:
        line = process.stdout.readline()
        if not line:
            break
        yield line.decode('UTF-8').strip()

### Un ejemplo de uso de la función anterior

In [18]:
dt, T = 0.001, 2

graphics = False
ff_interp= "FreeFem++" if graphics else "FreeFem++-nw"

command = "{} -ne keller-segel.edp -dt {} -nt {}".format(ff_interp,dt,T)

for line in run(command):
    print(line)

-- FreeFem++ v  3.380001 (date Sat Feb  6 20:00:25 UTC 2016)
Load: lg_fem lg_mesh lg_mesh3 eigenvalue
sizestack + 1024 =8000  ( 6976 )

Solving, iter: 1 (t=0.001)
max(u): 342.161
max(v): 140.233
Solving, iter: 2 (t=0.002)
max(u): 396.142
max(v): 139.722


### Otro ejemplo, filtrando la salida 

In [24]:
import re
for line in run(command):
    line = re.findall(r'iter .+$', line)
    if line:
        print(line[0].split()[1])

### Ejemplo final, barra de progreso

In [45]:
import ipywidgets as widgets
import re

dt, nt = 0.001, 20
T = dt*nt
graphics = False
ff_interp= "FreeFem++" if graphics else "FreeFem++-nw"
command = "{} -ne keller-segel.edp -dt {} -nt {}".format(ff_interp,dt,nt)

progress_bar = widgets.IntProgress(
    min=0,
    max=nt,
    description='Time:' )
display(progress_bar)
for line in run(command):
    line = re.findall(r'iter: .+$', line)
    if line:
        iteration = int(line[0].split()[1])
        print(iteration, end=" ")
        progress_bar.value = iteration
        progress_bar.description = "{}/{} iter".format(iteration, progress_bar.max)

IntProgress(value=0, description='Time:', max=20)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 